In [ ]:
def filter_clusters(clusters):
    result = []
    for cluster in clusters:
        new_cluster = []
        counts = [(el,cluster['nodes'].count(el)) for el in set(cluster['nodes'])]
        for count in counts:
            new_cluster.append(count[0]) if count[1] > 13 else None
                
        result.append({'nodes':new_cluster})
    
    with open('data/node_clusters_v3_filtered.pkl', 'wb') as outp:
        pickle.dump(result, outp, pickle.HIGHEST_PROTOCOL)
    
    return result

# clusters_v3_filtered = filter_clusters(clusters_v3)


In [ ]:
def cluster_with_count(clusters):
    result = []
    for cluster in clusters:
        new_cluster = []
        counts = [(el,cluster['nodes'].count(el)) for el in set(cluster['nodes'])]
        for count in counts:
            new_cluster.append(count)
                
        result.append({'nodes':new_cluster})
    
    with open('data/node_clusters_v3_with_count.pkl', 'wb') as outp:
        pickle.dump(result, outp, pickle.HIGHEST_PROTOCOL)
    
    return result

# clusters_v3_with_count = cluster_with_count(clusters_v3)


In [ ]:
def get_node_cluster_mapping(clusters):
    mapping = {}
    for i, node in enumerate(graph['nodes']):
        print('%.2f%%' % ((i * 100) / len(graph['nodes']))) if i % 500 == 0 else None

        node_clusters = [unique_labels[i] for i,cluster in enumerate(clusters) if node in cluster['nodes']]
        mapping[node] = node_clusters if len(node_clusters) > 0 else 'None'
                    
    with open('data/node_cluster_mapping_v2.pkl', 'wb') as outp:
        pickle.dump(mapping, outp, pickle.HIGHEST_PROTOCOL)

    return mapping
    
# node_cluster_mapping_v3 = get_node_cluster_mapping(clusters_v3_filtered)

In [7]:
def cluster_count(cluster,node):
    return [n[1] for n in cluster['nodes'] if n[0] == node][0]

def get_node_cluster_mapping_with_count(clusters):
    mapping = {}
    for i, node in enumerate(graph['nodes']):
        print('%.2f%%' % ((i * 100) / len(graph['nodes']))) if i % 500 == 0 else None

        node_clusters = [(unique_labels[i],cluster_count(cluster,node)) for i,cluster in enumerate(clusters) if node in [c[0] for c in cluster['nodes']]]
        node_clusters = sorted(node_clusters, key=lambda c: c[1], reverse=True)
        mapping[node] = node_clusters if len(node_clusters) > 0 else 'None'
                            
    with open('data/node_cluster_mapping_v3_with_count.pkl', 'wb') as outp:
        pickle.dump(mapping, outp, pickle.HIGHEST_PROTOCOL)

    return mapping

# node_cluster_mapping_v3_with_count = get_node_cluster_mapping_with_count(clusters_v3_with_count)

In [ ]:
def get_graph_clusters():
    clusters = []
    for i,d in enumerate(data):
        print('%.2f%%' % ((i * 100) / len(data))) if i % 10 == 0 else None
        nodes, edges, edge_types = [], [], []
        for dd in d['data']:
            graph = dd['graph']
            for node in graph['nodes']:
                nodes.append(node)
            
            for edge, edge_type in zip(graph['edges'], graph['edge_types']):
                if not any(edge['source'] == e['source'] and edge['target'] == e['target'] for e in edges):
                    edges.append({'source': edge['source'], 'target': edge['target']}) 
                    edge_types.append(edge_type)
        
                                       
        clusters.append({'nodes': nodes, 'edges': edges, 'edge_types': edge_types})
        
    with open('data/node_clusters_v3.pkl', 'wb') as outp:
        pickle.dump(clusters, outp, pickle.HIGHEST_PROTOCOL)
                                       
    return clusters

graph_embedding_size=100

def get_embedding(G):
        walk_length = 10
        rw = BiasedRandomWalk(G)
        walks = rw.run(
            nodes=G.nodes(),  # root nodes
            length=walk_length,  # maximum length of a random walk
            n=10,  # number of random walks per root node
            p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
            q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
            weighted=False,  # for weighted random walks
            seed=42,  # random seed fixed for reproducibility
        )

        model = Word2Vec(
            walks,  vector_size=graph_embedding_size, window=5, min_count=0, sg=1, workers=1
        )

        return model.wv.vectors
    
    
def get_embeddings(clusters):
    all_embeddings = []
    for i, cluster in clusters:
        edges_ = pd.DataFrame({
                'source': [e['source'] for e in cluster['edges']],
                'target': [e['target'] for e in cluster['edges']],
                'type': cluster['edge_types']
            })

        G = StellarGraph(IndexedArray(index=graph['nodes']), edges_, edge_type_column="type")

        node_embeddings = get_embedding(G)
        
        all_embeddings.append(node_embeddings)
    
    with open('data/node_embeddings_v4.pkl', 'wb') as outp:
        pickle.dump(all_embeddings, outp, pickle.HIGHEST_PROTOCOL)
        
    return all_embeddings

In [ ]:
graph_embedding_size = 100

def get_embedding(G):
        walk_length = 10
        rw = BiasedRandomWalk(G)
        walks = rw.run(
            nodes=G.nodes(),  # root nodes
            length=walk_length,  # maximum length of a random walk
            n=10,  # number of random walks per root node
            p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
            q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
            weighted=False,  # for weighted random walks
            seed=42,  # random seed fixed for reproducibility
        )

        model = Word2Vec(
            walks,  vector_size=graph_embedding_size, window=5, min_count=0, sg=1, workers=1
        )

        return model.wv.vectors
    
def get_graph():
    nodes, edges, edge_types = [], [], []
    for i1,d in enumerate(data):
        print('%.2f%%' % ((i1 * 100) / len(data))) if i1 % 10 == 0 else None
        for dd in d['data']:
            g = dd['context_graph']
            for node in g['nodes']:
                nodes.append(node) if not node in nodes else None

            for i, edge in enumerate(g['edges']):
                if not any(e['source'] == edge['source'] and e['target'] == edge['target'] for e in edges):
                    edges.append({'source': edge['source'], 'target': edge['target']})
                    edge_types.append(g['edge_types'][i])
    
    
    with open('data/graph_v2.pkl', 'wb') as outp:
        pickle.dump({'nodes': nodes, 'edges': edges, 'edge_types': edge_types}, outp, pickle.HIGHEST_PROTOCOL)
        
    return {'nodes':nodes, 'edges': edges, 'edge_types': edge_types}

def get_embeddings():
    graph = get_graph()
    # graph = unpickler('.','graph.pkl')

    edges_ = pd.DataFrame({
            'source': [e['source'] for e in graph['edges']],
            'target': [e['target'] for e in graph['edges']],
            'type': graph['edge_types']
        })
    
    G = StellarGraph(IndexedArray(index=graph['nodes']), edges_, edge_type_column="type")

    node_embeddings = get_embedding(G)
    
    with open('data/node_embeddings_v4.pkl', 'wb') as outp:
        pickle.dump(node_embeddings, outp, pickle.HIGHEST_PROTOCOL)
        
    return node_embeddings, graph

node_embeddings_v4, graph_v2 = get_embeddings()

In [10]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Art-Painting'],['Science-Biology'],['Science-Medicine']]
enc.fit(X)
encodings = enc.transform([['Art-Painting'],['Science-Biology']]).toarray()
multi_hot_encoding = np.zeros(3)
for enc in encodings:
    for i,val in enumerate(enc):
        multi_hot_encoding[i] = 1 if multi_hot_encoding[i] == 1 or val == 1 else 0

multi_hot_encoding

array([1., 1., 0.])